## MLflow Model Management

In [14]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [15]:
# create imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=142)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [16]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1422)

### Logistic Regression Classifier

In [17]:
log_reg = LogisticRegression(C=1, solver="liblinear")
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       270
           1       1.00      0.93      0.97        30

    accuracy                           0.99       300
   macro avg       1.00      0.97      0.98       300
weighted avg       0.99      0.99      0.99       300



### Random Forest Classifier

In [18]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       270
           1       1.00      0.93      0.97        30

    accuracy                           0.99       300
   macro avg       1.00      0.97      0.98       300
weighted avg       0.99      0.99      0.99       300



### XGBoost

In [19]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       270
           1       1.00      0.97      0.98        30

    accuracy                           1.00       300
   macro avg       1.00      0.98      0.99       300
weighted avg       1.00      1.00      1.00       300



### Handle class imbalance using SMOTETomek and train XGBoost

In [20]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([627, 627]))

In [21]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       270
           1       0.85      0.97      0.91        30

    accuracy                           0.98       300
   macro avg       0.92      0.97      0.95       300
weighted avg       0.98      0.98      0.98       300



In [22]:
classification_report(y_test, y_pred_xgb, output_dict=True)

{'0': {'precision': 0.9962406015037594,
  'recall': 0.9814814814814815,
  'f1-score': 0.9888059701492538,
  'support': 270.0},
 '1': {'precision': 0.8529411764705882,
  'recall': 0.9666666666666667,
  'f1-score': 0.90625,
  'support': 30.0},
 'accuracy': 0.98,
 'macro avg': {'precision': 0.9245908889871738,
  'recall': 0.9740740740740741,
  'f1-score': 0.9475279850746269,
  'support': 300.0},
 'weighted avg': {'precision': 0.9819106590004422,
  'recall': 0.98,
  'f1-score': 0.9805503731343284,
  'support': 300.0}}

### Track Experiments using MLFlow

In [23]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    ),
]

In [24]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [25]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [28]:
# initialize MLflow
mlflow.set_experiment("MLflow Model Management")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        
        mlflow.log_metric("accuracy", report['accuracy'])
        mlflow.log_metric("recall_class_1", report["1"]["recall"])
        mlflow.log_metric("recall_class_0", report["0"]["recall"])
        mlflow.log_metric("f1_score_macro", report["macro avg"]["f1-score"])

        ## Or
        # mlflow.log_metrics({
        #     'accuracy': report['accuracy'],
        #     'recall_class_1': report['1']['recall'],
        #     'recall_class_0': report['0']['recall'],
        #     'f1_score_macro': report['macro avg']['f1-score']
        # })
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/03/06 17:16:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/928107604711575783/runs/63605be38d524540ad1e97a406801fe3
🧪 View experiment at: http://localhost:5000/#/experiments/928107604711575783


2025/03/06 17:16:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/928107604711575783/runs/eb370fff18c14a10afbc0466149f6d47
🧪 View experiment at: http://localhost:5000/#/experiments/928107604711575783


2025/03/06 17:16:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/928107604711575783/runs/a17c7c92f7ac4f97b78584dcdb4bd8a4
🧪 View experiment at: http://localhost:5000/#/experiments/928107604711575783


2025/03/06 17:16:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/928107604711575783/runs/eb2471c9750549fdbe3456ac772917ad
🧪 View experiment at: http://localhost:5000/#/experiments/928107604711575783


### Register the model

In [40]:
model_name = "XGB-Smote"
run_id = input("Enter Run ID:")
model_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri, model_name
    # "runs:/d16076a3ec534311817565e6527539c0/sklearn-model", "sk-learn-random-forest-reg"
)

Enter Run ID: eb2471c9750549fdbe3456ac772917ad


Successfully registered model 'XGB-Smote'.
2025/03/06 18:26:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


### Checking if the model is registered correctly in MLflow

In [41]:
model_name = "XGB-Smote"
models = mlflow.search_registered_models(filter_string=f"name='{model_name}'")
print(models)

[<RegisteredModel: aliases={}, creation_timestamp=1741264889570, description='', last_updated_timestamp=1741264889633, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1741264889633, current_stage='None', description='', last_updated_timestamp=1741264889633, name='XGB-Smote', run_id='eb2471c9750549fdbe3456ac772917ad', run_link='', source='mlflow-artifacts:/928107604711575783/eb2471c9750549fdbe3456ac772917ad/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='XGB-Smote', tags={}>]


### Load the model

In [42]:
model_name = "XGB-Smote"
model_version = 1
# model_uri = f"models:/{reg_model_name}/1"
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 1, 0])

### Model to Production

In [43]:
from mlflow import MlflowClient

client = MlflowClient()
client.copy_model_version(
    src_model_uri=f"models:/{model_name}/{model_version}",
    dst_name="anomaly-detection-production",
)

Successfully registered model 'anomaly-detection-production'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-production'.


<ModelVersion: aliases=[], creation_timestamp=1741265302652, current_stage='None', description='', last_updated_timestamp=1741265302652, name='anomaly-detection-production', run_id='eb2471c9750549fdbe3456ac772917ad', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [45]:
model_name = "XGB-Smote"
model_version = 1
# model_uri = f"models:/{reg_model_name}/1"
model_uri = f"models:/anomaly-detection-production/1"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 1, 0])